# 🧠 Crowd Anomaly Detection – Preprocessing Notebook
This notebook handles preprocessing for the CUHK Avenue and Violent-Flows datasets by extracting frames, converting them to grayscale, resizing them, and generating label files.


## 📚 Import Required Libraries


In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.io import loadmat
from PIL import Image

## 🎞️ Frame Extraction Function
Extract frames from any video, convert to grayscale, resize to 200×200, and save them as PNG images.


In [2]:
def extract_frames_from_video(video_path, output_dir, resize_dim=(200, 200)):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, resize_dim)
        path = os.path.join(output_dir, f"frame_{idx:04d}.png")
        cv2.imwrite(path, resized)
        idx += 1
    cap.release()
    return idx  # nombre total de frames extraites


## 🧪 Process CUHK Avenue Test Set
Extract test video frames and assign labels using the .mat annotation files.


In [3]:
def process_avenue_test_set(test_video_dir, test_vol_dir, output_frames_dir, output_csv_path):
    os.makedirs(output_frames_dir, exist_ok=True)
    video_list = sorted([v for v in os.listdir(test_video_dir) if v.endswith('.avi')])
    data = []

    for i, video_file in enumerate(video_list):
        video_name = os.path.splitext(video_file)[0]
        video_path = os.path.join(test_video_dir, video_file)
        mat_path = os.path.join(test_vol_dir, f"vol{i+1:02d}.mat")

        print(f"🔄 Traitement de la vidéo : {video_file}")

        output_dir = os.path.join(output_frames_dir, video_name)
        n_frames = extract_frames_from_video(video_path, output_dir)

        mat = loadmat(mat_path)
        volume = mat['vol']
        n_vol_frames = volume.shape[2]
        min_len = min(n_frames, n_vol_frames)

        frame_labels = [1 if np.any(volume[:, :, f] == f) else 0 for f in range(min_len)]

        for j in range(min_len):
            frame_path = os.path.join(output_dir, f"frame_{j:04d}.png")
            if os.path.exists(frame_path):
                data.append({'path': frame_path, 'label': frame_labels[j]})

    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    print(f"✅ Labels enregistrés dans : {output_csv_path}")


## 🚀 Run Test Set Processing (Avenue)


In [10]:
process_avenue_test_set(
    test_video_dir='data/avenue/testing_videos/',
    test_vol_dir='data/avenue/testing_vol/',
    output_frames_dir='data/avenue/test_frames/',
    output_csv_path='data/avenue/test_labels.csv'
)


🔄 Traitement de la vidéo : 01.avi
🔄 Traitement de la vidéo : 02.avi
🔄 Traitement de la vidéo : 03.avi
🔄 Traitement de la vidéo : 04.avi
🔄 Traitement de la vidéo : 05.avi
🔄 Traitement de la vidéo : 06.avi
🔄 Traitement de la vidéo : 07.avi
🔄 Traitement de la vidéo : 08.avi
🔄 Traitement de la vidéo : 09.avi
🔄 Traitement de la vidéo : 10.avi
🔄 Traitement de la vidéo : 11.avi
🔄 Traitement de la vidéo : 12.avi
🔄 Traitement de la vidéo : 13.avi
🔄 Traitement de la vidéo : 14.avi
🔄 Traitement de la vidéo : 15.avi
🔄 Traitement de la vidéo : 16.avi
🔄 Traitement de la vidéo : 17.avi
🔄 Traitement de la vidéo : 18.avi
🔄 Traitement de la vidéo : 19.avi
🔄 Traitement de la vidéo : 20.avi
🔄 Traitement de la vidéo : 21.avi
✅ Labels enregistrés dans : data/avenue/test_labels.csv


## 🏗️ Process CUHK Avenue Training Set
Extract training video frames and label them all as normal (label = 0).


In [11]:
def process_avenue_training_set(training_video_dir, output_frames_dir, output_csv_path):
    os.makedirs(output_frames_dir, exist_ok=True)
    video_list = sorted([v for v in os.listdir(training_video_dir) if v.endswith('.avi')])
    data = []

    for video_file in video_list:
        video_name = os.path.splitext(video_file)[0]
        video_path = os.path.join(training_video_dir, video_file)

        print(f"🎞️ Extraction de : {video_file}")
        output_dir = os.path.join(output_frames_dir, video_name)
        os.makedirs(output_dir, exist_ok=True)

        cap = cv2.VideoCapture(video_path)
        idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (200, 200))
            frame_path = os.path.join(output_dir, f"frame_{idx:04d}.png")
            cv2.imwrite(frame_path, resized)
            data.append({'path': frame_path, 'label': 0})  # Tout est normal
            idx += 1
        cap.release()

    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    print(f"✅ Labels sauvegardés dans : {output_csv_path}")


## 🚀 Run Training Set Processing (Avenue)


In [12]:
process_avenue_training_set(
    training_video_dir='data/avenue/training_videos/',
    output_frames_dir='data/avenue/train_frames/',
    output_csv_path='data/avenue/train_labels.csv'
)

🎞️ Extraction de : 01.avi
🎞️ Extraction de : 02.avi
🎞️ Extraction de : 03.avi
🎞️ Extraction de : 04.avi
🎞️ Extraction de : 05.avi
🎞️ Extraction de : 06.avi
🎞️ Extraction de : 07.avi
🎞️ Extraction de : 08.avi
🎞️ Extraction de : 09.avi
🎞️ Extraction de : 10.avi
🎞️ Extraction de : 11.avi
🎞️ Extraction de : 12.avi
🎞️ Extraction de : 13.avi
🎞️ Extraction de : 14.avi
🎞️ Extraction de : 15.avi
🎞️ Extraction de : 16.avi
✅ Labels sauvegardés dans : data/avenue/train_labels.csv


## 🔥 Process Violent-Flows Dataset
Automatically extract and label frames from all videos in the Violent-Flows dataset based on directory structure (violent / non-violent).


In [15]:
def preprocess_violent_flows_dataset(root_dir, output_dir, output_csv):
    os.makedirs(output_dir, exist_ok=True)
    data = []
    folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
    folders.sort()

    for folder in folders:
        folder_path = os.path.join(root_dir, folder)
        for subdir, _, video_files in os.walk(folder_path):
            video_files = [f for f in video_files if f.endswith('.avi') or f.endswith('.mp4')]
            print(f"📂 Dossier : {subdir} - {len(video_files)} vidéos")

            for video_file in tqdm(video_files):
                video_path = os.path.join(subdir, video_file)
                label = 1 if 'violence' in subdir.lower() and 'non' not in subdir.lower() else 0

                video_name = os.path.splitext(video_file)[0]
                frame_output = os.path.join(output_dir, video_name)
                os.makedirs(frame_output, exist_ok=True)

                cap = cv2.VideoCapture(video_path)
                idx = 0
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    resized = cv2.resize(gray, (200, 200))
                    frame_path = os.path.join(frame_output, f"frame_{idx:04d}.png")
                    cv2.imwrite(frame_path, resized)
                    data.append({'path': frame_path, 'label': label})
                    idx += 1
                cap.release()

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV généré : {output_csv} - {len(df)} entrées")


## 🚀 Run Violent-Flows Processing


In [17]:
preprocess_violent_flows_dataset(
    root_dir='data/Violent-Flows',
    output_dir='data/Violent-Flows/frames',
    output_csv='data/Violent-Flows/violentflows_labels.csv'
)


📂 Dossier : data/Violent-Flows\1 - 0 vidéos


0it [00:00, ?it/s]


📂 Dossier : data/Violent-Flows\1\NonViolence - 25 vidéos


100%|██████████| 25/25 [00:08<00:00,  3.03it/s]


📂 Dossier : data/Violent-Flows\1\Violence - 25 vidéos


...

📂 Dossier : data/Violent-Flows\frames\stadium_violence__soccer_riot_compilation__qqcolaman__URwiZLZipeg - 0 vidéos


0it [00:00, ?it/s]


📂 Dossier : data/Violent-Flows\frames\stadium_violence__stadium_violence__Barras_Argentinas__alebb77__en5llVbjq20 - 0 vidéos


0it [00:00, ?it/s]

✅ CSV généré : data/Violent-Flows/violentflows_labels.csv - 22074 entrées


****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

## 🚶‍♀️ Preprocessing UCSD Anomaly Dataset (v1p2)
This section extracts test frames and detects anomaly labels using the corresponding ground truth masks provided in `_gt` folders.


In [18]:
# ⚙️ Function for processing UCSD Test (with masks)
def preprocess_ucsd_test(ped_dir, output_dir, output_csv, resize=(200, 200)):
    test_dir = os.path.join(ped_dir, 'Test')
    os.makedirs(output_dir, exist_ok=True)
    data = []

    test_folders = [f for f in os.listdir(test_dir)
                    if os.path.isdir(os.path.join(test_dir, f)) and not f.endswith('_gt')]

    for test_folder in sorted(test_folders):
        frame_folder = os.path.join(test_dir, test_folder)
        mask_folder = frame_folder + '_gt'
        has_mask = os.path.exists(mask_folder)

        print(f"\n📂 Processing test folder: {test_folder}")

        frame_files = sorted([
            f for f in os.listdir(frame_folder)
            if f.lower().endswith(('.tif', '.bmp', '.png')) and not f.startswith('.') and not f.startswith('._')
        ])

        for frame_file in tqdm(frame_files):
            frame_path = os.path.join(frame_folder, frame_file)
            save_folder = os.path.join(output_dir, test_folder)
            os.makedirs(save_folder, exist_ok=True)

            try:
                frame_img = Image.open(frame_path).convert('L').resize(resize)
            except Exception as e:
                print(f"⚠️ Error loading {frame_path}: {e}")
                continue

            save_path = os.path.join(save_folder, frame_file.replace('.tif', '.png'))
            frame_img.save(save_path)

            label = 0
            if has_mask:
                frame_name = os.path.splitext(frame_file)[0]
                mask_candidates = [
                    f for f in os.listdir(mask_folder)
                    if f.startswith(frame_name) and not f.startswith('.') and not f.startswith('._')
                ]
                if mask_candidates:
                    mask_path = os.path.join(mask_folder, mask_candidates[0])
                    try:
                        mask_img = Image.open(mask_path).convert('L').resize(resize)
                        mask_array = np.array(mask_img)
                        if np.any(mask_array == 255):
                            label = 1
                    except Exception as e:
                        print(f"⚠️ Error loading mask {mask_path}: {e}")

            data.append({'path': save_path, 'label': label})

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"✅ Test CSV saved to {output_csv} with {len(df)} entries.")

In [19]:
# ⚙️ Function for processing UCSD Train (all normal frames)
def preprocess_ucsd_train(ped_dir, output_dir, output_csv, resize=(200, 200)):
    train_dir = os.path.join(ped_dir, 'Train')
    os.makedirs(output_dir, exist_ok=True)
    data = []

    train_folders = [f for f in os.listdir(train_dir)
                     if os.path.isdir(os.path.join(train_dir, f)) and not f.startswith('.')]

    for train_folder in sorted(train_folders):
        folder_path = os.path.join(train_dir, train_folder)
        frame_files = sorted([
            f for f in os.listdir(folder_path)
            if f.lower().endswith(('.tif', '.bmp', '.png')) and not f.startswith('.') and not f.startswith('._')
        ])

        print(f"\n📂 Processing train folder: {train_folder}")

        for frame_file in tqdm(frame_files):
            frame_path = os.path.join(folder_path, frame_file)
            save_folder = os.path.join(output_dir, train_folder)
            os.makedirs(save_folder, exist_ok=True)

            try:
                frame_img = Image.open(frame_path).convert('L').resize(resize)
            except Exception as e:
                print(f"⚠️ Error loading {frame_path}: {e}")
                continue

            save_path = os.path.join(save_folder, frame_file.replace('.tif', '.png'))
            frame_img.save(save_path)

            data.append({'path': save_path, 'label': 0})

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"✅ Train CSV saved to {output_csv} with {len(df)} entries.")

## 🚀 Run preprocessing for UCSDped1 and UCSDped2


In [20]:
preprocess_ucsd_test(
    ped_dir='data/UCSD_Anomaly_Dataset.v1p2/UCSDped1',
    output_dir='data/UCSD_Anomaly_Dataset.v1p2/frames_ped1_test',
    output_csv='data/UCSD_Anomaly_Dataset.v1p2/labels_ped1_test.csv'
)

preprocess_ucsd_train(
    ped_dir='data/UCSD_Anomaly_Dataset.v1p2/UCSDped1',
    output_dir='data/UCSD_Anomaly_Dataset.v1p2/frames_ped1_train',
    output_csv='data/UCSD_Anomaly_Dataset.v1p2/labels_ped1_train.csv'
)

preprocess_ucsd_test(
    ped_dir='data/UCSD_Anomaly_Dataset.v1p2/UCSDped2',
    output_dir='data/UCSD_Anomaly_Dataset.v1p2/frames_ped2_test',
    output_csv='data/UCSD_Anomaly_Dataset.v1p2/labels_ped2_test.csv'
)

preprocess_ucsd_train(
    ped_dir='data/UCSD_Anomaly_Dataset.v1p2/UCSDped2',
    output_dir='data/UCSD_Anomaly_Dataset.v1p2/frames_ped2_train',
    output_csv='data/UCSD_Anomaly_Dataset.v1p2/labels_ped2_train.csv'
)


📂 Processing test folder: Test001


100%|██████████| 200/200 [00:23<00:00,  8.56it/s]



📂 Processing test folder: Test002


100%|██████████| 200/200 [00:09<00:00, 21.10it/s]



📂 Processing test folder: Test003


...


📂 Processing train folder: Train015


100%|██████████| 150/150 [00:05<00:00, 29.35it/s]



📂 Processing train folder: Train016


100%|██████████| 150/150 [00:07<00:00, 19.97it/s]

✅ Train CSV saved to data/UCSD_Anomaly_Dataset.v1p2/labels_ped2_train.csv with 2550 entries.
